In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")

In [3]:
df

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
0,2020-01-04 00:00:09,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
1,2020-01-04 00:00:41,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
2,2020-01-04 00:01:20,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
3,2020-01-04 00:04:12,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
4,2020-01-04 00:15:19,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
...,...,...,...,...,...,...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755411,2022-01-03 23:58:28,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755412,2022-01-03 23:58:46,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755413,2022-01-03 23:59:16,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True


In [4]:
df_3 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg({"registered_customer": ("count","sum"),"holiday":"mean","workingday":"mean"}).reset_index()

In [5]:
df_3

datetime registered_customer       holiday workingday
                             count   sum    mean       mean
0   2020-01-04                 985   654     0.0    0.00000
1   2020-01-05                 801   670     0.0    0.00000
2   2020-01-06                1349  1229     0.0    1.00000
3   2020-01-07                1562  1454     0.0    1.00000
4   2020-01-08                1600  1518     0.0    1.00000
..         ...                 ...   ...     ...        ...
726 2021-12-30                4231  4024     0.0    1.00000
727 2021-12-31                8144  7629     0.0    1.00000
728 2022-01-01                5932  5293     0.0    0.00000
729 2022-01-02                5182  4392     0.0    0.00000
730 2022-01-03                6302  5927     0.0    0.99762

[731 rows x 5 columns]

In [10]:
df_3[(df_3.workingday["mean"] != 1)&(df_3.workingday["mean"] != 0)]

datetime registered_customer         holiday workingday
                             count   sum      mean       mean
28  2020-02-01                2106  1870  0.000000   0.000950
56  2020-02-29                3339  2694  0.000000   0.001797
58  2020-03-02                2254  2054  0.000000   0.997782
65  2020-03-09                2106  1827  0.000000   0.998101
119 2020-05-02                6207  4145  0.000000   0.000483
121 2020-05-04                5920  4678  0.000000   0.999831
126 2020-05-09                5229  3438  0.000000   0.000191
150 2020-06-02                7059  5964  0.000142   0.999858
154 2020-06-06                6499  4278  0.000000   0.000154
182 2020-07-04                7055  4207  0.000000   0.000283
240 2020-08-31                8666  6942  0.000000   0.999423
248 2020-09-08                3321  3038  0.000301   0.999699
273 2020-10-03                4494  3581  0.000000   0.000445
301 2020-10-31                4920  4353  0.000000   0.003455
303 2020-11-02                5934  5477  0.000000   0.999494
308 2020-11-07                4651  3345  0.000000   0.000215
310 2020-11-09                4474  4056  0.000000   0.999776
331 2020-11-30                7538  6611  0.000000   0.998541
336 2020-12-05                4602  3718  0.000000   0.000217
394 2021-02-01                5748  5224  0.000000   0.997738
422 2021-03-01                6815  6025  0.000000   0.998826
427 2021-03-06                5008  3907  0.000000   0.000200
457 2021-04-05                7848  6221  0.000000   0.999745
483 2021-05-01               10031  6456  0.000000   0.000897
490 2021-05-08                7839  5045  0.000000   0.000128
514 2021-06-01               10951  9352  0.000274   0.999726
520 2021-06-07                8347  7042  0.000000   0.999760
546 2021-07-03                9467  6219  0.000000   0.000106
548 2021-07-05                8221  7026  0.000000   0.999878
576 2021-08-02               10877  8891  0.000000   0.999081
583 2021-08-09                7871  6509  0.000000   0.999873
609 2021-09-04                8804  5672  0.000000   0.000114
637 2021-10-02               12188  8034  0.000000   0.000082
639 2021-10-04                9519  8350  0.000000   0.999895
667 2021-11-01                3476  3450  0.000000   0.998849
674 2021-11-08                5994  5534  0.000000   0.999833
730 2022-01-03                6302  5927  0.000000   0.997620

In [11]:
df[df.datetime.between("2020-2-1", "2020-2-2")]

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
34320,2020-02-01 00:00:25,0.0,1.0,"clear, few clouds",9.84,11.365,60.0,15.0013,True
34321,2020-02-01 00:01:11,0.0,1.0,"clear, few clouds",9.84,11.365,60.0,15.0013,True
34322,2020-02-01 00:01:21,0.0,0.0,"clear, few clouds",9.02,9.850,64.0,23.9994,False
34323,2020-02-01 00:02:18,0.0,0.0,"clear, few clouds",9.02,9.850,64.0,23.9994,True
34324,2020-02-01 00:02:24,0.0,0.0,"clear, few clouds",9.02,9.850,64.0,23.9994,True
...,...,...,...,...,...,...,...,...,...
36421,2020-02-01 23:57:26,0.0,0.0,"clear, few clouds",6.56,9.850,80.0,6.0032,True
36422,2020-02-01 23:58:21,0.0,0.0,"clear, few clouds",6.56,9.850,80.0,6.0032,True
36423,2020-02-01 23:58:49,0.0,0.0,"clear, few clouds",6.56,9.850,80.0,6.0032,True
36424,2020-02-01 23:59:07,0.0,0.0,"clear, few clouds",6.56,9.850,80.0,6.0032,True
